In [11]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')  # Без графического интерфейса
import matplotlib.pyplot as plt

d, q = map(float, input("Введите d, q --> ").split())
e = 0.001  

a = [
    [q, 1, 0, 0, 0],
    [1, -2, 1, 0, 0],
    [0, 1, -2, 1, 0],
    [0, 0, 1, -2, 1],
    [0, 0, 0, 1, q]
]

b = [0, d, d, d, 0]


Введите d, q --> 1 -3.17


t --> первые значения x (предыдущие)

In [12]:
t = [0 for _ in range(len(b))]

accuracy = [float("inf")] * len(b)
history_accuracy = []  
iterations = 0  

vr --> временное x (фактическое)
идем по каждой строке вычисляем сумму Xj*aij кроме диагонального значения(его мы ищем)
потом из-за того что переносим иксы в другую сторону от = иеняеи знак на минус => 
      то что было - то что считали выше / коэфф X которого ищем
считаем точность по модулю и присваеваем t то что получили  

In [13]:
while max(accuracy) > e:
    vr = [0] * len(b)

    for i in range(len(a)):
        num = sum(a[i][j] * t[j] for j in range(len(a[0])) if j!= i)  

        vr[i] = (b[i]-num)/a[i][i]

    accuracy = [abs(vr[i] - t[i]) for i in range(len(t))]
    history_accuracy.append(max(accuracy))  
    t = vr
    iterations += 1

print("{:>15} {:>10} {:>10} {:>10} {:>10} {:>10}".format("Итерационный",*t))

   Итерационный -0.6899550004919165 -2.188089901203555 -2.6871573515593754 -2.188089901203555 -0.6899550004919165


No comment NUMPY

In [14]:
A=np.array(a)
B=np.array(b)
x= np.linalg.solve(A,B)
print("{:>10} {:>10} {:>10} {:>10} {:>10} {:>10}".format("numpy solver",*x))

numpy solver -0.6912442396313364 -2.1912442396313363 -2.6912442396313363 -2.1912442396313363 -0.6912442396313364


#9мЕтод Зейгея:
+ vr сразу присваемваем t и тогда умножаем на vr[i] и сразу же изменяем vr[i]

In [15]:
accuracy = [float("inf")] * len(b)
history_accuracy_1 = []  
iterations_1 = 0  

t = [0] * len(b)
while max(accuracy) > e:
    vr = t.copy()

    for i in range(len(a)):
        num = sum(a[i][j] * vr[j] for j in range(len(a)) if j != i)  
        vr[i] = (b[i] - num) / a[i][i]  

    accuracy = [abs(vr[i] - t[i]) for i in range(len(t))]
    history_accuracy_1.append(max(accuracy))  
    if max(accuracy) < e:
        break

    t = vr
    iterations_1 += 1

print("{:>20} {:>10} {:>10} {:>10} {:>10} {:>10}".format("Зейгеля",*t))


             Зейгеля -0.6901020903412296 -2.1888628583613476 -2.688862858361311 -2.1896779368086166 -0.6907501377945163


#9метод гейгеля + релаксация
тоже самое что и гейгеля но когда считаем точность 
заранее выбрав наиболее оптимальный коэффициент релаксации

In [16]:
t = [0] * len(b)

accuracy = [float("inf")] * len(b)
history_accuracy_2 = []  
iterations_2 = 0  
w = 1.4
while max(accuracy) > e:
    vr = t.copy()

    for i in range(len(a)):
        num = sum(a[i][j] * vr[j] for j in range(len(a)) if j != i)  
        vr[i] = (1-w)*t[i]+w*(b[i] - num) / a[i][i]  

    accuracy = [abs(vr[i] - t[i]) for i in range(len(t))]
    history_accuracy_2.append(max(accuracy))  
    if max(accuracy) < e:
        break

    t = vr
    iterations_2 += 1
print("{:>20} {:>10} {:>10} {:>10} {:>10} {:>10}".format("Зейгеля + релаксация",*t))

maxxx = max(iterations_1,iterations)
if min(len(history_accuracy_1),len(history_accuracy)) == len(history_accuracy):
    for i in range(len(history_accuracy),maxxx):
        history_accuracy.append(-0.1)
else:
    for i in range(len(history_accuracy_1),maxxx):
        history_accuracy_1.append(-0.1)

plt.figure(figsize=(8, 5))
if len(history_accuracy_2)<=maxxx:
    for i in range(len(history_accuracy_2),maxxx):
        history_accuracy_2.append(-0.1)
    plt.plot(range(1, maxxx + 1), history_accuracy_2, marker='o', linestyle='-', color='g', label="Зейгеля + релаксация")

plt.plot(range(1, maxxx + 1), history_accuracy_1, marker='o', linestyle='-', color='r', label="Зейгеля")
plt.plot(range(1, maxxx + 1), history_accuracy, marker='o', linestyle='-', color='b', label="Итерацонный")
plt.xlabel("Номер итерации")
plt.ylabel("Максимальная ошибка")
plt.title("Зависимость точности от числа итераций")
plt.grid()
plt.legend()
plt.show()

Зейгеля + релаксация -0.6911843650551275 -2.191302463062619 -2.691400798264625 -2.1914463065483547 -0.6913006046924617


Итерационный + релаксация

проверяем коэффициенты релаксации от 0.2 до 2.0 с шагом 0.2

если мы используем метод итерации + метод релаксации, то желательно использовать w до 1, так как при большем значениии возможен вариант перепрыгнуть нужный результат и уйти в inf

In [9]:
w = 0.2
while w<2.1:
    t = [0] * len(b)

    accuracy = [float("inf")] * len(b)
    history_accuracy = []  
    iterations = 0  

    while max(accuracy) > e:
        vr = [0] * len(b)

        for i in range(len(a)):
            num = sum(a[i][j] * t[j] for j in range(len(a[0])) if j!= i)  
            vr[i] = (1-w)*t[i] + w*(b[i]-num)/a[i][i]

        accuracy = [abs(vr[i] - t[i]) for i in range(len(t))]
        history_accuracy.append(max(accuracy))  
        t = vr
        iterations += 1
    print(w,iterations)
    w = round(w+0.2,2)
    print(*t)

0.2 121
-0.6833892270176948 -2.1710499050436023 -2.666343849645153 -2.1710499050436023 -0.6833892270176948
0.4 69
-0.6875766175601732 -2.1818152048369583 -2.6796178776657493 -2.1818152048369583 -0.6875766175601732
0.6 49
-0.6889601435167305 -2.185372091051019 -2.6840036549480355 -2.185372091051019 -0.6889601435167305
0.8 38
-0.6896049136349258 -2.1870297209160525 -2.686047576222715 -2.1870297209160525 -0.6896049136349258
1.0 31
-0.6899550004919165 -2.188089901203555 -2.6871573515593754 -2.188089901203555 -0.6899550004919165
1.2 4459
nan -inf inf -inf nan
1.4 1662
nan inf -inf inf nan
1.6 1113
nan -inf inf -inf nan
1.8 875
nan nan inf nan nan
2.0 740
nan inf -inf inf nan


Зейгеля + релаксация с шагом 0.2

In [10]:
w = 0.2
while w<2:
    t = [0] * len(b)

    accuracy = [float("inf")] * len(b)
    history_accuracy = []  
    iterations = 0  

    while max(accuracy) > e:
        vr = t.copy()

        for i in range(len(a)):
            num = sum(a[i][j] * vr[j] for j in range(len(a)) if j != i)  
            vr[i] = (1-w)*t[i]+w*(b[i] - num) / a[i][i]  

        accuracy = [abs(vr[i] - t[i]) for i in range(len(t))]
        history_accuracy.append(max(accuracy))  
        if max(accuracy) < e:
            break

        t = vr
        iterations += 1
    print(w,iterations)
    w = round(w+0.2,2)
    print(*t)

0.2 112
-0.6834169690477538 -2.171539836653447 -2.667453380351899 -2.1723511164665372 -0.6840482369712649
0.4 58
-0.6875285230802646 -2.182134821928864 -2.680533176519485 -2.182960597586356 -0.6881716539608456
0.6 37
-0.6889953740218485 -2.1859197489136535 -2.685197983028793 -2.1867283651271614 -0.689626565317405
0.8 25
-0.689659558198576 -2.1876717840297566 -2.687381587057079 -2.1884972865012142 -0.6903073017663919
1.0 17
-0.6901020903412296 -2.1888628583613476 -2.688862858361311 -2.1896779368086166 -0.6907501377945163
1.2 11
-0.6906926700066678 -2.190282163134907 -2.690439461481872 -2.190801474197085 -0.6911274060363677
1.4 10
-0.6911843650551275 -2.191302463062619 -2.691400798264625 -2.1914463065483547 -0.6913006046924617
1.6 16
-0.6913784015596698 -2.1909186573134702 -2.690656173506617 -2.1907265779567178 -0.6910052773152674
1.8 33
-0.6909997180320825 -2.192069910452049 -2.6928711345509857 -2.192182240381148 -0.691647230521149
